In [ ]:
"""
# SR15 Employment Sub-Regional Allocation
Last Updated: 03/28/23
@author: Michael Ma
"""
import pandas as pd
import numpy as np
import sqlalchemy as sql

## Employment Sub-Regional Allocation Model (Regiona Level to MGRA Level)

### Module 1 (EDD Job Distribution)

In [ ]:
# Get edd jobs total
edd_sectors_mgra = pd.read_csv('Employment CT_MGRA RAW_DATA 03232023.csv')
edd_sectors_mgra = pd.melt(edd_sectors_mgra, 
                           id_vars=['CT20','mgra'], 
                           value_vars=edd_sectors_mgra.columns[2:])
edd_sectors_mgra[['industry','sector_num']] = edd_sectors_mgra['variable'].str.split('_',1,expand=True)
edd_sectors_mgra = edd_sectors_mgra[['CT20',
                                     'mgra',
                                     'sector_num',
                                     'value']]

# Get the sectors control total from Naiomi
sectors = pd.read_csv('sectors.csv')
sectors[['sector','sector_num','industry']] = sectors['industry'].str.split('_',2,expand=True)
sectors = sectors[['industry',
                   'sector_num',
                   'control_total']]

# Get the name for each sector
sectors_crosswalk = sectors[['industry',
                             'sector_num']].copy()
edd_sectors_mgra_1 = pd.merge(edd_sectors_mgra, 
                              sectors_crosswalk, 
                              how ='left',
                              on='sector_num')[['CT20',
                                                'mgra',
                                                'industry',
                                                'value']]
edd_sectors_mgra_2 = edd_sectors_mgra_1.pivot(index=['CT20','mgra'],
                                              columns ='industry',
                                              values='value').reset_index()

# Pivot back to sector
sectors = pd.pivot_table(sectors,
                         values='control_total', 
                         columns='industry')

# Control to EDD Job Total
col_1 = sectors.columns[0:]
col_2 = edd_sectors_mgra_2.columns[2:]

for c1, c2 in zip(col_1, col_2):
    edd_sectors_mgra_2[c2] = edd_sectors_mgra_2[c2] * ((sectors[c1].values[0]) / 
                                                       (edd_sectors_mgra_2[c2].sum()))

# Keep only the job sectors we want
edd_sectors_distribution = edd_sectors_mgra_2[['mgra',
                                               'Accommodation and food services',
                                               'Arts, entertainment, and recreation',
                                               'Educational services; private', 
                                               'Farm, Forestry, fishing, and hunting',
                                               'Federal', 
                                               'Finance and insurance',
                                               'Management of companies and enterprises',
                                               'Manufacturing', 
                                               'Mining',
                                               'Real estate and rental and leasing', 
                                               'Retail trade',
                                               'State and Local Government',
                                               'Utilities', 
                                               'Wholesale trade']]

# Check each sector's total
sum_cols_edd = edd_sectors_distribution.columns[1:]
col_sum_edd = edd_sectors_distribution[sum_cols_edd].sum()

# Check the table and number
# display(col_sum_edd)
# edd_sectors_distribution

### Module 2 (Home Base, using ACS job distribution)

In [ ]:
'''
Importing Table
'''
B24080Region = "select yr, geoid, geo_name ,table_description, line_number, line_desc, estimate \
                        from census.acs.vw_summary_file \
                        where release_type='5Y' and \
                              summary_level=050 and \
                              subject_table = 'B24080' and \
                              yr = 2021 and \
                              county = 073 \
                        order by line_number"

B24070Region = "select yr, geoid, geo_name ,table_description, line_number, line_desc, estimate \
                        from census.acs.vw_summary_file \
                        where release_type='1Y' and \
                              subject_table ='B24070' and \
                              yr = 2021 and \
                              county = 073 \
                        order by line_number"

B24080Tract = "select yr, geoid, tract, table_description, line_number, line_desc, estimate \
                        from census.acs.vw_summary_file \
                        where release_type='5Y' and \
                              summary_level=140 and \
                              subject_table='B24080' and \
                              yr = 2021 \
                        order by tract, line_number"

engine = sql.create_engine('mssql+pymssql://DDAMWSQL16/')
B24080Tract = pd.read_sql(B24080Tract, 
                          engine.connect())
B24080Region = pd.read_sql(B24080Region, 
                           engine.connect())
B24070Region = pd.read_sql(B24070Region, 
                           engine.connect())


'''
Get the regional total self-employed from ACS
'''
B24080Group = B24080Tract[B24080Tract['line_number'].isin([5,10,11,15,20,21])]
B24080Group = B24080Group.groupby(['tract']).sum()[['estimate']].reset_index()
B24080Group = B24080Group.assign(category='Self-employed')
B24080Group['estimate'].sum()


'''
Take the B24070Region Table to get each sector's distribution
'''
# self_employed_incorporated
self_employed = [57,58,59,60,61,62,64,65,66,68,69,71,72,73,75,76,78,79,80,81,138,139,
                140,141,142,143,145,146,147,149,150,152,153,154,156,157,159,160,161,162]

B24070Region_selfemployed = B24070Region[(B24070Region['line_number'].isin(self_employed))]
B24070Region_selfemployed = B24070Region_selfemployed.groupby(['line_desc']).sum()[['estimate']].reset_index()
B24070Region_selfemployed['pct'] = B24070Region_selfemployed['estimate'] / B24070Region_selfemployed['estimate'].sum()
B24070Region_selfemployed = B24070Region_selfemployed.assign(category='Self-employed')                       
B24070Region_selfemployed = B24070Region_selfemployed[['category',
                                                       'line_desc',
                                                       'pct','estimate']]


'''  
Merging with B24080 table to get each sectors in each census tract 
'''
new_tract_job = pd.merge(B24080Group, 
                         B24070Region_selfemployed, 
                         left_on='category', 
                         right_on='category', 
                         how='left')
new_tract_job['self_employed'] = new_tract_job['estimate_x'] * new_tract_job['pct']
new_tract_job = new_tract_job[['tract',
                               'line_desc',
                               'self_employed']]


'''   
Pivot the jobs to columns
'''
new_sector_pivoted = pd.pivot_table(new_tract_job, 
                                    values='self_employed', 
                                    index='tract', 
                                    columns='line_desc', 
                                    aggfunc=np.sum).reset_index()
new_sector_pivoted = new_sector_pivoted.set_index('tract')
new_sector_pivoted['total_jobs'] = new_sector_pivoted.sum(axis=1)
new_sector_pivoted.columns.name = ''


'''
Get the sectors that use the ACS distribution   
'''

sectors = pd.read_csv('sectors.csv')
sectors[['sector','sector_num','industry']] = sectors['industry'].str.split('_',2,expand=True)
sectors = sectors[['industry',
                   'sector_num',
                   'control_total']]
# Pivot back to sector
sectors = pd.pivot_table(sectors,
                         values='control_total', 
                         columns='industry')
sectors = sectors[['Information',
                   'Professional, scientific, and technical services',
                   'Administrative and support and waste management services']]

# Get the sectors that use the ACS distribution
new_sector_acs = new_sector_pivoted[['Information',
                                     'Professional, scientific, and technical services',
                                     'Administrative and support and waste management services']].reset_index()
# Scale each sector total to Naiomi's REMI control total
for column in new_sector_acs.columns[1:]:
    new_sector_acs[column] = new_sector_acs[column] * \
                             (sectors[column][0] / new_sector_acs[column].sum())

new_sector_acs['tract'] = new_sector_acs['tract'].astype(int)
new_sector_homebase = new_sector_acs.set_index('tract')
new_sector_homebase['total_jobs'] = new_sector_homebase.sum(axis=1)

# Get each sector's share in each census tract (ACS distribution)
for column in new_sector_homebase.columns[:-1]:
    new_sector_homebase[column] = new_sector_homebase[column] / new_sector_homebase['total_jobs']

# Each sector's distribution in census tract and the total jobs
new_sector_homebase = new_sector_homebase.reset_index()
new_sector_homebase['tract'] = new_sector_homebase['tract'].astype(int)


'''
1. Get the working age pop (18 to 64) 
2. Get each mgra's share of population in each census tract
'''
# Working age population
pop2021 = ''' SELECT mgra,
	               [pop_18to19],
	               [pop_20to24],
	               [pop_25to29],
                     [pop_30to34],
                     [pop_35to39],
                     [pop_40to44],
                     [pop_45to49],
                     [pop_50to54],
                     [pop_55to59],
                     [pop_60to61],
                     [pop_62to64]
                     FROM [concep_test].[concep_2022].[detailed_pop_tab_mgra] 
                     WHERE ethnicity=0 and 
                           estimates_year = 2021 
                     ORDER BY mgra'''
pop2021 = pd.read_sql(pop2021, 
                      engine.connect())

# get the total working age population
pop2021_mgra = pop2021.set_index('mgra')
pop2021_mgra['total_pop'] = pop2021_mgra.sum(axis=1)
pop2021_mgra = pop2021_mgra[['total_pop']] 
pop2021_mgra = pop2021_mgra.reset_index()

# Tract 2020 crosswalk MGRA
ct20mgra15 = ''' SELECT CT20, MGRA as mgra
                        FROM [GeoDepot].[gis].[MGRA15]
                        ORDER BY CT20, 
                                 mgra'''
engineb8 = sql.create_engine('mssql+pymssql://SQL2014B8/')
ct20mgra15 = pd.read_sql(ct20mgra15, 
                         engineb8.connect())

# Get the total population by Census Tract and MGRA 15
# Get the total population for each census tract
# get the percentage of mgra pop in corresponded census tract
ct_mgra_pop = pd.merge(pop2021_mgra,
                       ct20mgra15, 
                       on='mgra')[['CT20',
                                   'mgra',
                                   'total_pop']]
ct_mgra_pop = ct_mgra_pop.sort_values(by=['CT20'])
ct_pop = ct_mgra_pop.groupby(by=['CT20']).sum()[['total_pop']].reset_index()
pct_ct_mgra_pop = pd.merge(ct_mgra_pop,
                           ct_pop,
                           how='left',
                           on='CT20')
pct_ct_mgra_pop['pct_ct_mgra_pop'] = pct_ct_mgra_pop['total_pop_x'] / pct_ct_mgra_pop['total_pop_y']
pct_ct_mgra_pop = pct_ct_mgra_pop.rename(columns={'CT20':'tract',
                                                  'total_pop_x':'mgra_pop',
                                                  'total_pop_y':'tract_pop'})


'''    
Allocating Tract Jobs to MGRA
'''

# Total jobs in the sector
total_jobs_homebase_tract = new_sector_homebase[['tract',
                                                 'total_jobs']]
# Get the total self-employed jobs in the mgra 
job_homebase_mgra = pd.merge(total_jobs_homebase_tract,
                             pct_ct_mgra_pop,
                             on='tract',
                             how='right')
job_homebase_mgra['total_jobs_mgra'] = job_homebase_mgra['total_jobs'] * job_homebase_mgra['pct_ct_mgra_pop']
job_homebase_mgra = job_homebase_mgra[['tract',
                                       'tract_pop',
                                       'mgra',
                                       'mgra_pop',
                                       'total_jobs',
                                       'total_jobs_mgra']].copy()

# Get each job number in the mgra. Get table from new_sector_homebase (The table that has the pct of each sector in each ct)
final_homebase = pd.merge(job_homebase_mgra,
                          new_sector_homebase,
                          how='left',
                          on='tract')
final_homebase = final_homebase[['mgra',
                                 'total_jobs_mgra',
                                 'Information',
                                 'Professional, scientific, and technical services',
                                 'Administrative and support and waste management services']].copy()
# Get the each sector's job
job_homebase_sector = final_homebase.columns[2:]
for column in final_homebase[job_homebase_sector]:
    final_homebase[column] = final_homebase[column] * final_homebase['total_jobs_mgra']

acshomebase_sectors_distribution = final_homebase[['mgra',
                                                   'Information',
                                                   'Professional, scientific, and technical services',
                                                   'Administrative and support and waste management services']].sort_values(['mgra']).copy()

# Check each sector's total
sum_cols_homebase = acshomebase_sectors_distribution.columns[1:]
col_sum_homebase = acshomebase_sectors_distribution[sum_cols_homebase].sum()
# display(col_sum_homebase)
# display(acshomebase_sectors_distribution)

### Module 3 (Everywhere, equally distribute job to each MGRA that has jobs rehardless of the sectors)

In [ ]:
# Get edd jobs total
everywhere_sectors_mgra = pd.read_csv('Employment CT_MGRA RAW_DATA 03232023.csv')
everywhere_sectors_mgra = pd.melt(everywhere_sectors_mgra, 
                           id_vars=['CT20','mgra'], 
                           value_vars=everywhere_sectors_mgra.columns[2:])

everywhere_sectors_mgra[['industry','sector_num']] = everywhere_sectors_mgra['variable'].str.split('_',1,expand=True)
everywhere_sectors_mgra = everywhere_sectors_mgra[['CT20',
                                     'mgra',
                                     'sector_num',
                                     'value']]

# Get the sectors control total from Naiomi
sectors = pd.read_csv('sectors.csv')
sectors[['sector','sector_num','industry']] = sectors['industry'].str.split('_',2,expand=True)
sectors = sectors[['industry',
                   'sector_num',
                   'control_total']]

# Get the name for each sector
sectors_crosswalk = sectors[['industry',
                             'sector_num']].copy()
everywhere_sectors_mgra_1 = pd.merge(everywhere_sectors_mgra, 
                              sectors_crosswalk, 
                              how ='left',
                              on='sector_num')[['CT20',
                                                'mgra',
                                                'industry',
                                                'value']]
everywhere_sectors_mgra_2 = everywhere_sectors_mgra_1.pivot(index=['CT20','mgra'],
                                              columns ='industry',
                                              values='value').reset_index()

# Keep only the sectors that use the everywhere distribution
everywhere_sectors_mgra_3 = everywhere_sectors_mgra_2[['mgra',
                                                       'Construction',
                                                       'Transportation and warehousing',
                                                       'Health care and social assistance',
                                                       'Other services, except public administration']]

# get the share of each mgra out of all mgra (we are assuming the jobs we keep are present in all mgra equally)
everywhere_sectors_mgra_3['everywhere_pct'] = 1 / everywhere_sectors_mgra_3['mgra'].count()

# Pivot back to sector
sectors = pd.pivot_table(sectors,
                         values='control_total', 
                         columns='industry')

# Keep only the sectors we need for the everywhere distribution
sectors = sectors[['Construction',
                   'Transportation and warehousing',
                   'Health care and social assistance',
                   'Other services, except public administration']]

# Apply the share of each mgra to the sectors (we should get eqaul jobs total for each mgra for each sector)
col1 = everywhere_sectors_mgra_3.columns[1:-1]
col2 = sectors.columns[0:]
for c1, c2 in zip(col1, col2):
    # print(c1,'_',c2)
    everywhere_sectors_mgra_3[c1] = everywhere_sectors_mgra_3['everywhere_pct'] * sectors[c2][0]

everywhere_sectors_distribution = everywhere_sectors_mgra_3.copy()


sum_cols_everywhere = everywhere_sectors_distribution.columns[1:-1]
col_sum_everywhere = everywhere_sectors_distribution[sum_cols_everywhere].sum()
# display(everywhere_sectors_distribution)
# display(col_sum_everywhere)

### Final module 4-1 (rounding error adjustment)

In [ ]:
# Join the first two tables
edd_everywhere = pd.merge(edd_sectors_distribution, 
                          everywhere_sectors_distribution, 
                          on='mgra')[['mgra',
                                    'Accommodation and food services',
                                    'Arts, entertainment, and recreation',
                                    'Educational services; private', 
                                    'Farm, Forestry, fishing, and hunting',
                                    'Federal', 
                                    'Finance and insurance',
                                    'Management of companies and enterprises',
                                    'Manufacturing', 
                                    'Mining',
                                    'Real estate and rental and leasing', 
                                    'Retail trade',
                                    'State and Local Government',
                                    'Utilities', 
                                    'Wholesale trade',
                                    'Construction',
                                    'Transportation and warehousing',
                                    'Health care and social assistance',
                                    'Other services, except public administration']]
# Second join to the third table
edd_acshomebase_everywhere = pd.merge(acshomebase_sectors_distribution,
                                      edd_everywhere,
                                      on='mgra',
                                      how='left').fillna(0)

# Order by MGRA
edd_acshomebase_everywhere = edd_acshomebase_everywhere.sort_values(by=['mgra'])
# edd_acshomebase_everywhere.to_csv('job_sectors_mgra15.csv')


# Rounding error adjustment
# Get the columns needed for rounding
all_sectors = edd_acshomebase_everywhere.columns[1:]
# Get the index column
job = edd_acshomebase_everywhere.reset_index()
rounded_error_adj = []
for column in job[all_sectors]:

    # 1. Get the rounded column for each sector
    job[f'{column}_rounded'] = job[column].round()
    # 2. Rank the unrounded number for each column(sector)
    job[f'{column}_rank'] = job[column].rank(method='first', ascending=False)
    
    job = job.sort_values(by=[f'{column}_rank'])
    # 3. Get the rounded total and the original total, take the difference
    originaltotal = job.sum()[column]
    rounded = job.sum()[f'{column}_rounded']
    x = (rounded - originaltotal).round().astype(int)
    # print(x)
    
    # 4. Assigning extra or insufficient to the rounded column based on the rank
    if x > 0:
        idx = job['index'].iloc[:x].tolist()
        job.loc[job["index"].isin(idx), f'{column}_rounded'] -= 1
    if x < 0:
        x = x * -1
        idx = job['index'].iloc[:x].tolist()
        job.loc[job["index"].isin(idx), f'{column}_rounded'] += 1

    job = job.drop(columns={column,f'{column}_rank'})

rounded_error_adj.append(job)

final_job = pd.concat(rounded_error_adj).sort_values(by=['mgra'])
final_job = final_job.drop(columns={'index'})
# Clean up column names
final_job.columns.values[1:] = final_job.columns[1:].str.split('_').str[0]
final_job.to_csv('non_wage_s_allocation_mgra.csv',index=False)

# Check the total for each sector
sum_cols_full = final_job.columns[1:]
col_sum_full = final_job[sum_cols_full].sum()

### Final Module 4-2 (Crosswalk to jurisdiction to get jurisdiction total)

In [ ]:
## Crosswalk to jurisdiction to get the jurisdiction total
# Get jurisdiction mgra crosswalk
engineB8 = sql.create_engine('mssql+pymssql://SQL2014B8/')
mgra15_city = '''SELECT [MGRA],
                        [City] 
                   FROM [GeoDepot].[gis].[MGRA15]'''
mgra15_city = pd.read_sql(mgra15_city, 
                          engineB8.connect())

# Get city id and city name
engine = sql.create_engine('mssql+pymssql://DDAMWSQL16/')
City_id = '''SELECT distinct jurisdiction, 
                        jurisdiction_id 
                   FROM [demographic_warehouse].[dim].[mgra_denormalize] 
                   WHERE series = 14 
                   ORDER BY jurisdiction_id'''
City_id = pd.read_sql(City_id, engine.connect())

# Merge two tables to tag jurisdiction to mgra
MGRA15_City = pd.merge(mgra15_city,
                       City_id, 
                       left_on='City',
                       right_on='jurisdiction_id')[['MGRA',
                                                    'jurisdiction']]

# Total Pop by MGRA
pop_2022_concep = '''SELECT mgra, 
                            pop 
                       FROM [concep_test].[concep_2022].[detailed_pop_tab_mgra] 
                       WHERE ethnicity = 0 AND 
                             estimates_year = 2022 
                       ORDER BY mgra'''
pop_2022_concep = pd.read_sql(pop_2022_concep, 
                              engine.connect())
# Merge the MGRA pop with city to get cities' total pop
Pop_City_Mgra = pd.merge(pop_2022_concep,
                         MGRA15_City, 
                         left_on='mgra',
                         right_on='MGRA')[['MGRA',
                                           'jurisdiction',
                                           'pop']]
Pop_City_Mgra = Pop_City_Mgra.groupby(['jurisdiction']).sum()[['pop']].reset_index()
Pop_City_Mgra.to_csv('concep_pop_2022.csv')

# Merge to get jurisdiction total
jurisdiction_jobs = pd.merge(MGRA15_City,
                             final_job, 
                             left_on='MGRA', 
                             right_on='mgra')
jurisdiction_jobs = jurisdiction_jobs.groupby(['jurisdiction']).sum()
jurisdiction_jobs = jurisdiction_jobs.drop(columns={'MGRA','mgra'})

# Get the column sum
sum = jurisdiction_jobs.sum()
sum.name = 'All_sectors'
jurisdiction_jobs = jurisdiction_jobs.append(sum.transpose())
# Get the row sum
sum_cols_final = jurisdiction_jobs.columns[0:]
jurisdiction_jobs['Total_jobs'] = jurisdiction_jobs[sum_cols_final].sum(axis=1)

# Export the jurisdiction level
jurisdiction_jobs.to_csv('non_wage_s_allocation_jurisdiction.csv')